<a href="https://colab.research.google.com/github/choeuneheol/python-practice/blob/master/%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0%EC%B1%85DAY6~7(%EB%B2%94%EC%A3%BC%ED%98%95_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#베이스라인 모델
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
suvimission = pd.read_csv('sample_submission.csv')

In [2]:
all_data = pd.concat([train, test])         # 훈련 데이터와 테스트 데이터 합치기
all_data = all_data.drop('target', axis=1)  # 타깃값 제거
all_data

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,499995,0,0,0,F,N,Green,Square,Lion,Canada,...,9e4b23160,acc31291f,1,Novice,Lava Hot,j,A,Gb,1,3
199996,499996,1,0,0,F,Y,Green,Trapezoid,Lion,China,...,cfbd87ed0,eae3446d0,1,Contributor,Lava Hot,f,S,Ed,2,2
199997,499997,0,1,1,T,Y,Green,Trapezoid,Lion,Canada,...,1108bcd6c,33dd3cf4b,1,Novice,Boiling Hot,g,V,TR,3,1
199998,499998,1,0,0,T,Y,Blue,Star,Hamster,Costa Rica,...,606ac930b,d4cf587dd,2,Grandmaster,Boiling Hot,g,X,Ye,2,1


In [3]:
#원-핫 인코딩

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()                             # 원-핫 인코더 생성
all_data_encoded = encoder.fit_transform(all_data)    # 원-핫 인코딩 적용

In [4]:
# 데이터 나누기

num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
x_train = all_data_encoded[:num_train]    # 0~num_train ~ 1행
x_test = all_data_encoded                 # num_train~마지막행

y=train['target']

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

In [6]:
#모델 훈련

from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42) 
logistic_model.fit(x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [7]:
logistic_model.predict_proba(x_valid)

array([[0.23057724, 0.76942276],
       [0.91580546, 0.08419454],
       [0.84242849, 0.15757151],
       ...,
       [0.25273963, 0.74726037],
       [0.50728863, 0.49271137],
       [0.95770316, 0.04229684]])

In [8]:
logistic_model.predict(x_valid)

array([1, 0, 0, ..., 1, 0, 0])

In [9]:
# 검증 데이터를 활용한 타깃 예측
y_valid_preds = logistic_model.predict_proba(x_valid)[:,1]

In [10]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.7976


In [11]:
# 예측 및 결과 제출
# 타깃값 1일 확률 예측

y_preds = logistic_model.predict_proba(x_test)[:,1]

In [12]:
# submission['target'] = y_preds
# submission.to_csv('submission.csv')

In [13]:
#성능개선1

train2 = pd.read_csv('train.csv')
test2 = pd.read_csv('test.csv')
submisson2 = pd.read_csv('sample_submission.csv')

In [14]:
#데이터 합치기
#훈련 데이터와 테스트 데이터 합치기

all_data = pd.concat([train2, test2])
all_data = all_data.drop('target',axis=1) # 타깃값 제거

In [15]:
#이진 피처 인코딩

all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

In [16]:
#순서형 피처 인코딩
ord1dict = {'Novice':0, 'Contributor':1,
           'Expert':2, 'Master':3, 'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2,
            'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [17]:
from sklearn.preprocessing import OrdinalEncoder


In [19]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3','ord_4','ord_5']

ord_encoder = OrdinalEncoder()  # OrdinalEncoder 객체 생성

# ordinal 인코딩 적용
all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])

# 피처별 인코딩 순서 출력
for feature, categories in zip(ord_345, ord_encoder.categories_):
  print(feature)
  print(categories)

ord_3
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o']
ord_4
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
ord_5
['AP' 'Ai' 'Aj' 'BA' 'BE' 'Bb' 'Bd' 'Bn' 'CL' 'CM' 'CU' 'CZ' 'Cl' 'DH'
 'DN' 'Dc' 'Dx' 'Ed' 'Eg' 'Er' 'FI' 'Fd' 'Fo' 'GD' 'GJ' 'Gb' 'Gx' 'Hj'
 'IK' 'Id' 'JX' 'Jc' 'Jf' 'Jt' 'KR' 'KZ' 'Kf' 'Kq' 'LE' 'MC' 'MO' 'MV'
 'Mf' 'Ml' 'Mx' 'NV' 'Nf' 'Nk' 'OR' 'Ob' 'Os' 'PA' 'PQ' 'PZ' 'Ps' 'QM'
 'Qb' 'Qh' 'Qo' 'RG' 'RL' 'RP' 'Rm' 'Ry' 'SB' 'Sc' 'TR' 'TZ' 'To' 'UO'
 'Uk' 'Uu' 'Vf' 'Vx' 'WE' 'Wc' 'Wv' 'XI' 'Xh' 'Xi' 'YC' 'Yb' 'Ye' 'ZR'
 'ZS' 'Zc' 'Zq' 'aF' 'aM' 'aO' 'aP' 'ac' 'av' 'bF' 'bJ' 'be' 'cA' 'cG'
 'cW' 'ck' 'cp' 'dB' 'dE' 'dN' 'dO' 'dP' 'dQ' 'dZ' 'dh' 'eG' 'eQ' 'eb'
 'eg' 'ek' 'ex' 'fO' 'fh' 'gJ' 'gM' 'hL' 'hT' 'hh' 'hp' 'iT' 'ih' 'jS'
 'jV' 'je' 'jp' 'kC' 'kE' 'kK' 'kL' 'kU' 'kW' 'ke' 'kr' 'kw' 'lF' 'lL'
 'll' 'lx' 'mb' 'mc' 'mm' 'nX' 'nh' 'oC' 'oG' 'oH' 'oK' 'od' 'on' 'pa'
 'ps' 'qA' 'qJ' 'qK' 'qP' 'qX' '

In [20]:
nom_features = ['nom_' + str(i) for i in range(10)] #명목형 피처

In [21]:
# 정상 실행되지 않는 코드(오류발생)
from sklearn.preprocessing import OneHotEncoder

Onehot_encoder = OneHotEncoder()
all_data[nom_features] = Onehot_encoder.fit_transform(all_data[nom_features])

ValueError: ignored

In [23]:
#정상 실행되는 코드
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder() # OneHotEncoder 객체 생성

# 원-핫 인코딩 적용
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

encoded_nom_matrix

<500000x16276 sparse matrix of type '<class 'numpy.float64'>'
	with 5000000 stored elements in Compressed Sparse Row format>

In [24]:
all_data = all_data.drop(nom_features, axis=1) # 기존 명목형 피처 삭제

In [25]:
# 날짜 피처 인코딩
date_features = ['day','month'] #날짜 피처

# 원-핫 인코딩 적용
encoded_date_matrix=onehot_encoder.fit_transform(all_data[date_features])

all_data = all_data.drop(date_features, axis=1) # 기존 날짜 피처 삭제

encoded_date_matrix

<500000x19 sparse matrix of type '<class 'numpy.float64'>'
	with 1000000 stored elements in Compressed Sparse Row format>

In [27]:
# 피처 엔지니어링2 : 피처 스케일링

from sklearn.preprocessing import MinMaxScaler

ord_features = ['ord_'+ str(i) for i in range(6)] # 순서형 피처

# min-max 정규화
all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])

In [29]:
#인코딩 및 스케일링된 피처 합치기

from scipy import sparse

#인코딩 및 스케일링된 피처 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                               encoded_nom_matrix,
                               encoded_date_matrix],
                              format='csr')

all_data_sprs

<500000x16307 sparse matrix of type '<class 'numpy.float64'>'
	with 9663717 stored elements in Compressed Sparse Row format>

In [31]:
#데이터 나누기
num_train = len(train)                # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
x_train = all_data_sprs[:num_train]   # 0~num_train - 1행
x_test = all_data_sprs[num_train:]    # num_train ~ 마지막행

y = train['target']

In [32]:
from sklearn.model_selection import train_test_split

#훈련 데이터, 검증 데이터 분리
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

In [33]:
#하이퍼파라미터 최적화

%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#로지스틱 회귀 모델 생성
logistic_model = LogisticRegression()

#하이퍼파라미터 값 목록
lr_params = {'C':[0.1, 0.125, 0.2], 'max_iter':[800, 900, 1000],
             'solver':['liblinear'], 'random_state':[42]}

CPU times: user 36 µs, sys: 0 ns, total: 36 µs
Wall time: 41 µs


In [34]:
#그리드서치 객체 생성
gridsearch_logistic_model = GridSearchCV(estimator=logistic_model,
                                         param_grid = lr_params,
                                         scoring='roc_auc', # 평가지표
                                         cv=5)

#그리드서치 수행
gridsearch_logistic_model.fit(x_train, y_train)

print('최적 하이퍼파라미터:', gridsearch_logistic_model.best_params_)

최적 하이퍼파라미터: {'C': 0.1, 'max_iter': 800, 'random_state': 42, 'solver': 'liblinear'}


In [35]:
#모델 성능 검증

y_valid_preds = gridsearch_logistic_model.predict_proba(x_valid)[:,1]

In [36]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:4f}')

검증 데이터 ROC AUC : 0.503108


In [39]:
#예측 및 결과 제출
# 타깃값 1일 확률 예측
y_preds = gridsearch_logistic_model.best_estimator_.predict_proba(x_test)[:,1]

# 제출 파일 생성
